In [45]:
import numpy as np
import math
import random as rand
import cv2
import matplotlib.pyplot as plt

In [46]:
#This is to implement he gradient boosting algorithm as implemented in the Kazemi 1 ms face alignment file

In [47]:
def Open(loc):
    l = []
    file_ptr = open(loc, "r")
    image_name = file_ptr.readline()
    for x in file_ptr:
        k = x.split(',')
        k[1] = k[1].split('\n')[0]
        k[0] = float(k[0])
        k[1] = float(k[1])
        l.append(k)
    return image_name.split("\n")[0] , l

In [48]:
def Face_detection(img):
    #cnts = []
    img = cv2.imread(img)
    #copy = img.copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier('haarcascades\haarcascade_frontalface_default.xml')
    faces = faceCascade.detectMultiScale(gray,1.1,2)
    
    max_val = -1
    val = [0, 0, 0, 0]
    for (x,y,w,h) in faces:
        if w*h >= max_val:
            val[0] = x
            val[1] = y
            val[2] = w
            val[3] = h
            max_val = w*h
    
    #    cv2.rectangle(copy,(x,y),(x+w,y+h),(0,255,0),2)

    #for i in range(194):
    #    cv2.circle(copy,(int(k[index_k][i][0]),int(k[index_k][i][1])),2,(0,0,255),-1)

    #resized = imutils.resize(copy,width=500)  
    return val[0], val[1], val[2], val[3]

In [72]:
def main(N):
    TRAIN_DIR = 'train_1/'
    annotations = 'annotation/'
    GroundTruth = []
    Image_Vectors = []
    x0 = 0
    y0 = 0
    h0 = 0 
    w0 = 0 

    for i in range(1, N+1):
        print(i)
        img , gt = Open(annotations+str(i)+".txt")
        GroundTruth.append(gt)
        Image_Vectors.append(TRAIN_DIR + str(img) + ".jpg")
        x, y, h, w = Face_detection(Image_Vectors[i-1])
        x0 += x
        y0 += y
        h0 += h
        w0 += w
    
    Bounding_box = [x0/N, y0/N, h0/N, w0/N]
    Shape_estimate = driver(Image_Vectors, GroundTruth, Bounding_box)
main(2)

1
2
[309.86, 450.98] [-4.4934722609468736e-11, 1.9269918993813917e-11]


TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [64]:
def driver(Image_vectors, GroundTruth, Bounding_box, Number_of_strong_regression = 10):
    
    Shape_estimate = []
    Delta_S = []
    for i in range(len(Image_vectors)):
        index = rand.randint(0, len(Image_vectors)-1)
        while index == i:
            index = rand.randint(0, len(Image_vectors)-1)
        val = GroundTruth[index]
        Shape_estimate.append(val)
        for j in range(len(GroundTruth[0])):
            Delta_S.append([GroundTruth[i][j][0] - val[j][0] ,GroundTruth[i][j][1] - val[j][1]])
    learning_rate = 0.7
    
    while Number_of_strong_regression > 0:
        r = GradientBoost(Image_vectors, GroundTruth, Shape_estimate, Delta_S, len(Image_vectors), learning_rate, Bounding_box)
        for i in range(len(Image_vectors)):
            Shape_estimate[i] += r
            Delta_S[i] = (GroundTruth[i] - Shape_estimate[i])
        
        Number_of_strong_regression -= 1
    
    return Shape_estimate
    
        

In [71]:
def GradientBoost(Image_vectors, GroundTruth, Shape_estimate, target_Update_step, Number_of_training_samples, learning_rate, Bounding_box, Number_of_regressors = 20):
    
    '''
    Image_Vectors: dim(N, dim(of each image)): It is the set of all the images
    Shape_estimate: dim(N, dim(landmarks)): It is the set of all the landmarks 
    target_Update_step: dim(N, dim(landmarks)): It is the set of all pseudo-residuals
    Number_of_regressors(weak): K(20)
    Number_of_training_samples: n
    learning_rate: v
    '''
    

    
    #Initialise 
    S_temp  = [0]*len(target_Update_step[0])
    
    for i in range(len(target_Update_step)):
        for j in range(len(target_Update_step[i])):
            S_temp[j] += target_Update_step[i][j]
            
    f = [X/len(S_temp) for X in S_temp]
    
    
    #In loop
    for k in range(Number_of_regressors):
        
        r = []
        for i in range(Number_of_training_samples):
            print(target_Update_step[i] , f)
            r.append(target_Update_step[i] - f)
        
        ret = []
        fit_regressor(Image_vectors, GroundTruth, r, Shape_estimate, Mean_Shape_estimate, Image_vectors, ret, Bounding_box)
            
        f += learning_rate*np.mean(ret)
    
    return f
    
    
    

In [9]:
def fit_regressor(I, r, GroundTruth, Shape_estimate, Mean_Shape_estimate, Q, ret_Shape_Estimate, Bounding_box, levels = 5):
    
    """
    Q: it is the set of all images at current node before split occurs
    """
    
    
    #bucktes = []
    #splits = []
    if levels == 0:
        #Each image find S_pi_i and Subtract from Shape_estimate 
        #return Q
        delta_S = 0
        for img in Q:
            index = I.index(img)
            S_pi_i = GroundTruth[index]
            delta_S  += S_pi_i - Shape_estimate[index]
        
        ret_Shape_Estimate.append(delta_S/len(Q))
            
        return None
        
        
    Ql, Qr = get_Best_Split_for_Node(I, r, Q, Shape_estimate, Mean_Shape_estimate, box)
    fit_regressor(I, r, Shape_estimate, Mean_Shape_estimate, Ql, splits, Bounding_box, levels - 1)
    fit_regressor(I, r, Shape_estimate, Mean_Shape_estimate, Qr, splits, Bounding_box, levels - 1)
    
    #for l in levels:
    #    Ql1, Qr1, split_l =  get_Best_Split_for_Node(I, r, Ql, Shape_estimate, Mean_Shape_estimate)
    #    Ql, Qr, split_r = get_Best_Split_for_Node(I, r, Qr, Shape_estimate, Mean_Shape_estimate)

    

def get_Best_Split_for_Node(I, r, Q, Shape_estimate, Mean_Shape_estimate, box):
    Candidate_splits = get_Candidate_splits(I, box)
    for ele in Candidate_splits:
        s, r, t = SimilarityTransform(Shape_estimate, Mean_Shape_estimate) #change the func to get the mean from outside source
        
        for i in range(0, 2):
            closest_landmark = getClosestLandmark(Mean_Shape_estimate, ele[i])
            offset = [a-b for a, b in zip(ele, closest_landmark)]
            u_dash = getClosestLandmark(Shape_estimate, ele[i]) + (1/s)*np.transpose(r)*offset
            #new_u_v.append(u_dash)
            ele[i] = u_dash
        
    Ql = []
    Qr = []
    for split in Candidate_splits:
        left, right = check_Intensity(I, split)
        Ql.append(left)
        Qr.append(right)
        
    
    
    u = (1/len(Q))*np.sum(r)
    
    u_Ql = []
    for i in range(len(Ql)):
        for ele in Ql[i]:
            u_Ql[i] += r.index(ele)
        u_Ql[i] /= len(Ql[i])
        
    
    
    for i in range(len(u_Ql)):
        u_Qr[i] = (len(Q))*u - len(Ql[i])*u_Ql[i]
        u_Qr[i] = np.divide(u_Qr[i], Qr[i])
        
    E_Qtheta = []
    for i in range(len(u_Ql)):
        E_Qtheta.append(len(Ql)*np.dot(np.transpose(u_Ql), u_Ql) + len(Qr)*np.dot(np.transpose(u_Qr), u_Qr))
    
    best_split = E_Qtheta.index(max(E_Qtheta))
    
    return Ql[best_split], Qr[best_split]
     

In [10]:
def check_Intensity(I, theta):
    left = []
    right = []
    for img in I:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #gray[theta[0]][0]
        #gray[x][y]
        #theta[0][0] = u[0]
        if gray[theta[0][0]][theta[0][1]] - gray[theta[1][0]][theta[1][1]] > theta[2]:
            left.append(img)
        else:
            right.append(img)
    return left, right

In [11]:
def get_Closest_Landmark(Shape_Estimate, pix):
    
    min_val = 0xffffffff
    ret_pix
    for ele in Shape_estimate:
        dist = math.sqrt(sum(ele[0] - pix[0])**2 + (ele[1] - pix[1])**2)
        if dist <= min_val:
            min_val = dist
            ret_pix = ele
        
    return ret_pix
                
    

In [12]:
def get_Candidate_splits(I, box, S = 20):
    
    '''
    i/p: The set of all n images
    
    o/p: The set of candidate splits for each and every image i.e N dimension x dimension of each pixel
    
    '''
    
    Candidate_splits = []
    
    for i in range(S):
        pix = get_pixels(I, box)
        u = pix[randint(0, len(pix))]
        v = pix[randint(0, len(pix))]
        while not check_prior(u, v):
            u = pix[randint(0, len(pix))]
            v = pix[randint(0, len(pix))]
        thresh = np.random.uniform(-0.25, 0.25) 
        Candidate_splits.append([u, v, thresh])
    
    return Candidate_splits
        

In [13]:
def check_prior(u, v):
    dist = np.absolute(np.linalg.norm(u-v)) 
    prob = np.exp(-dist/.1) 
    if u != v and prob > np.random.random(): 
        return True
    return False
    

In [14]:
def get_pixels(I, Bounding_box, P = 400):
    
    """
    i/p: The bounding box generated for this particular image :- I
    (x0, y0): coordinates of the top left of the bounding box
    h, w: height and width of the bounding box respectively
    P: the number of pixel locations(400)
    
    o/p: set of P pixels in the bounding box
    """
    
    x0 = Bounding_box[0]
    y0 = Bounding_box[1]
    h = Bounding_box[2]
    w = Bounding_box[3]
    ret = []
    for i in range(P):
        ret_x = randint(x0, x0+w)
        ret_y = randint(y0, y0+h)
        ret.append([ret_x, ret_y])
    
    return ret
    